<a href="https://colab.research.google.com/github/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/TPMLGymGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning
## Appliqué à [Gym.OpenAI]((https://gym.openai.com)

Voir la page d'[introduction à Gym](https://github.com/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/TPMLIntroGym.ipynb).

### Installation de gym

#### Outil AUTRE QUE COLAB (pyzo, jupyter lab, .....)

In [ ]:
!pip install gym
!pip install Box2D


In [2]:
#GYM
import gym
import Box2D as b2

#Random
from random import  randint
#
from math import exp
#for sorting
import functools
#array
import numpy as np

#for charts
import matplotlib.pyplot as plt
#On MAc : 
#import matplotlib
#matplotlib.use("MacOSX")

#### <font color=red>Sous colab</font>
*Sous colab, l'affichage ne sera pas dynamique.. Il faudra passer soit par un rendu sous forme de plot (matplotlib), soit passer par une vidéo à enregistrer...*

***Solution pour affichage statique***<br>
*Il vous faut importer, seulement si vous utilisez colab, les packages suivants :*

In [ ]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install x11-utils

In [ ]:
#GYM
import gym
import Box2D as b2

#Random
from random import  randint
#
from math import exp
#for sorting
import functools
#array
import numpy as np

#for charts
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()

---
## Charger l' environnement
On utilise ici l'environnement `LunarLander-v2`.


In [3]:
env = gym.make('LunarLander-v2') 

---
### Les valeurs des caractéristiques
Pour `LunarLander-v2`, vous trouverez l'explication des valeurs [ici](https://gym.openai.com/envs/LunarLander-v2/).

Il s'agit de faire atterrir une sonde sur une planète.

L'espace des actions est une valeur discrète dans l'intervalle [0,3] : 
  - 0 : Ne rien faire
  - 1 : allumage du moteur gauche
  - 2 : allumage du moteur central
  - 3 : allumage du moteur droit

Les observations sont au nombre de 8 : 
  - obs[0] : coordonnée horizontale
  - obs[1] : coordonnée verticale
  - obs[2] : vitesse horizontale
  - obs[3] : vitesse verticale
  - obs[4] : angle de la sonde
  - obs[5] : vitesse angulaire (de rotation)
  - obs[6] : 1 si le pied gauche de la sonde est au contact du sol, 0 sinon
  - obs[7] : 1 si le pied droit de la sonde est au contact du sol, 0 sinon

Le but est donc de faire atterrir la sonde dans une position donnée (0,0) avec une vitesse suffisamment faible pour éviter le crash.


In [4]:
print(env.spec)
print("nombre et type d'actions = ", env.action_space) 
print("nombre et type d'observations = ", env.observation_space) 
print("valeurs de  observations les plus basses = ", env.observation_space.low) 
print("valeurs de observations les plus elevées = ", env.observation_space.high) 

EnvSpec(LunarLander-v2)
nombre et type d'actions =  Discrete(4)
nombre et type d'observations =  Box(-inf, inf, (8,), float32)
valeurs de  observations les plus basses =  [-inf -inf -inf -inf -inf -inf -inf -inf]
valeurs de observations les plus elevées =  [inf inf inf inf inf inf inf inf]


### (Re)Initialiser l'environnement
`env.reset()` initialise les variables d'environnement. L'exécution est obligatoire après avoir chargé l'environnement. Ici ce sont les coordonnées, l'angle et les vitesses.

C'est un environnement non déterministe par défaut, `env.reset()` retourne des valeurs légèrement différentes à chaque exécution : 

In [5]:
env.reset()

array([ 0.00223417,  1.4011867 ,  0.22626404, -0.43259206, -0.00258188,
       -0.05125214,  0.        ,  0.        ], dtype=float32)

L'algorithme génétique se prète mal aux environnement non stables; aussi nous utiliserons la fonction suivante pour fixer les vitesses et angles initiaux : 

In [6]:
def reset():
    env.reset();
    env.lander.angle=0
    env.lander.angularVelocity = 0
    env.lander.ApplyForceToCenter( (0,0,), False)
    env.lander.linearVelocity = b2.b2Vec2()

---
### Les actions
- Dans `LunarLander-v2` : [-1.0] fait reculer le véhicule d'une force 1, [0] le met au point mort, [1.0] le fait avancer d'une force 1

- ***Exécuter une action :***
  - *exécution* : `observation, reward,done,info = env.step(action)`
    - `observation`: ici les 8 éléments d'observation
    - `reward`: utilité de l'état (récompense 100 si bien posé, -100 si crash)
    - `done` : booléen (True si but (position) atteinte)
    - `info` : éventuelle information retournée par l'outil

**Exemples d'actions**

In [7]:
def test_action(no: int, nb: int = 1):
    """effectue nb fois l'action no dans l'environnement chargé"""
    for _ in range(nb):
        env.render()
        obs, reward, done, info = env.step(no)
        print(obs, reward, done, info)

def test():
    test_action(0, 40)
    test_action(2, 40)
    test_action(1, 40)
    test_action(3, 40)

In [ ]:
reset()
test()

---

# Test d'algorithme génétique
## Test de programme évolutionnaire pour atteindre l'objectif
- Utiliser l'environnement `LunarLander-v2`, car il est plus rapide 
- Programmez (en python) un algo génétique avec initialement les éléments suivants : 
  - sequence : tableau de valeurs entières 0, 1, 2 ou 3.
  - 1 point de croisement (au centre)
    - *(pour commencer, on pourrait proposer 2 points de croisement (au 1/3, et 2/3))*
  - taille sequence : taille_seq = 150
  - taille population initiale : taille_pop = 500
  - taux de gènes mutants dans une séquence : taux_mut_seq = 0.1
  - taux de séquences mutantes dans la population : taux_mut_pop = 0.2
  - nombre max de cycles de reproductions : nb_cycles = 3000


- Pour tester une séquence, il suffit de balayer le tableau et de lancer l'action correspondante..
- Les 4 meilleures séquences (11,22,33,44) se reproduisent entre elles, donnant donc 12 fils : 12, 13, 14, 21, 23, 24, 31, 32, 34, 41, 42, 43.
- Idéalement, un filtre doit permettre de supprimer les séquences identiques qui pourraient venir des croisements et des mutations.

- Tester votre algo sur le nb de générations et afficher le rendu toutes les 100 générations et bien sûr le rendu de la meilleure séquence finale.
  - (uniquement la position finale *colab*)
  - (sous forme d'animation sous autre outil)


- ***NB:*** 
  - Au plus simple il est possible d'utiliser une population (liste) de couples [sequence, utilite] où l'utilité serait la distance en x.

In [ ]:
#exemple :  
env.env.state= np.array([-0.5, 0])
tab = np.random.randint(-1, 2, 3) 
couple = [tab, 0]
print(couple)
for a in couple[0]:
    obs, reward, done, info = env.step([a])
print("arrive en x=", round(obs[0],2), ", fini=", done)
couple[1] = obs[0]
print("couple [actions, arrivee] = ", couple)

### Bonus : 
- Adapter la récompense pour favoriser les séquences qui atteignent le but en un minimum d'actions utiles (0 étant considéré comme une action inutile)